# The Big Mac Index Regression Model
This Notebook outlines all of the steps necessary to create a Regression Model using the price of a Big Mac in three different countries. The basic idea is that historical data is used to build a model that theoretically can be used to infer the price of a Big Mac in those three countries (including the U.S.).

The data source was taken from Kaggle: https://www.kaggle.com/datasets/mrmorj/big-mac-index-data

In [1]:
import pandas as pd

## Import Data
I have the data in Google Drive so I can easily get to it. Note that I already removed a bunch of columns from the original Kaggle dataset for simplicity. For example, I am not using the current variations for each country.

In [2]:
original_mac_df = pd.read_csv('../data/big mac.csv')

## Perform Exploratory Data Analysis
Start with the basics - what's in the dataset.

In [3]:
original_mac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1386 non-null   object 
 1   iso_a3         1386 non-null   object 
 2   currency_code  1386 non-null   object 
 3   name           1386 non-null   object 
 4   local_price    1386 non-null   float64
 5   dollar_ex      1386 non-null   float64
 6   dollar_price   1386 non-null   float64
dtypes: float64(3), object(4)
memory usage: 75.9+ KB


### Observations
The first four features require some engineering since they are objects. There are no missing values to deal with.

Next, output some of the rows to literally see the data.

In [4]:
original_mac_df.head(10)

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
0,4/1/2000,ARG,ARS,Argentina,2.50,1.000000,2.500000
1,4/1/2000,AUS,AUD,Australia,2.59,1.680000,1.541667
2,4/1/2000,BRA,BRL,Brazil,2.95,1.790000,1.648045
3,4/1/2000,CAN,CAD,Canada,2.85,1.470000,1.938776
4,4/1/2000,CHE,CHF,Switzerland,5.90,1.700000,3.470588
5,4/1/2000,CHL,CLP,Chile,1260.00,514.000000,2.451362
6,4/1/2000,CHN,CNY,China,9.90,8.280000,1.195652
7,4/1/2000,CZE,CZK,Czech Republic,54.37,39.100000,1.390537
8,4/1/2000,DNK,DKK,Denmark,24.75,8.040000,3.078358
9,4/1/2000,EUZ,EUR,Euro area,2.56,1.075269,2.380800


## Drop Columns You Don't Need
The `iso_a3`, `currency_code`, and `name` columns are essentially the same information. Going to ddrop `iso_a3` and `name`. Shorter text will be easier for the ML to understand (marginally better).

In [5]:
reduced_mac_df = original_mac_df.drop(columns=['iso_a3', 'name'])
reduced_mac_df.head(30)

,date,currency_code,local_price,dollar_ex,dollar_price
0,4/1/2000,ARS,2.50,1.000000,2.500000
1,4/1/2000,AUD,2.59,1.680000,1.541667
2,4/1/2000,BRL,2.95,1.790000,1.648045
3,4/1/2000,CAD,2.85,1.470000,1.938776
4,4/1/2000,CHF,5.90,1.700000,3.470588
5,4/1/2000,CLP,1260.00,514.000000,2.451362
6,4/1/2000,CNY,9.90,8.280000,1.195652
7,4/1/2000,CZK,54.37,39.100000,1.390537
8,4/1/2000,DKK,24.75,8.040000,3.078358
9,4/1/2000,EUR,2.56,1.075269,2.380800


## Feature Engineering
The `date` feature need to be converted to a timestamp integer and the `currency_code` needs to be one-hot encoded. The `currenct_code` feature is unordered (one country is not greater or less than another), which is why OHE is used. It will add many more columns.

In [6]:
from datetime import datetime

def create_timestamp(in_date: str) -> int:
  date_as_datetime = datetime.strptime(in_date, '%m/%d/%Y')
  return int(date_as_datetime.timestamp())

In [7]:
#To convert the date for all values in the DataFrame, I'm using a Lambda function.
reduced_mac_df['date'] = reduced_mac_df['date'].apply(lambda x: create_timestamp(x))

one_hot_encoded_df = pd.get_dummies(reduced_mac_df, columns = ['currency_code'])
one_hot_encoded_df.head()

,date,local_price,dollar_ex,dollar_price,currency_code_AED,currency_code_ARS,currency_code_AUD,currency_code_AZN,currency_code_BHD,currency_code_BRL,...,currency_code_SEK,currency_code_SGD,currency_code_THB,currency_code_TRY,currency_code_TWD,currency_code_UAH,currency_code_USD,currency_code_UYU,currency_code_VND,currency_code_ZAR
0,954547200,2.50,1.00,2.500000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,954547200,2.59,1.68,1.541667,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,954547200,2.95,1.79,1.648045,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,954547200,2.85,1.47,1.938776,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,954547200,5.90,1.70,3.470588,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Build the Model
There are many algorithms that will work, and some may be better suited for this example, but for simplicity and clarity, LinearRegression is used.

In [8]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Set the target and features - this establishes X and y.
y = one_hot_encoded_df['dollar_price']
X = one_hot_encoded_df[one_hot_encoded_df.columns.drop('dollar_price')]

# Split the data between training and test. Both are run with values (supervised learning). We can measure 'accuracy' using this approach.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train the model
linear_regression_model = linear_model.LinearRegression()
linear_regression_model.fit(X_train, y_train)

# Get the y-values for the training data, give it the
y_pred_train = linear_regression_model.predict(X_train)

# Now run the test data (note that instead of y_train and X_train, I gave it y_test and X_test)
y_pred_test = linear_regression_model.predict(X_test)

# Check if the model works - R2 or the coefficent of determination indicates what percentage of the y value comes from the x values. Higher is better.
# R2
print('Coefficient of determination (R2) TRAIN: %.2f'% r2_score(y_train, y_pred_train))
print('Coefficient of determination (R2) TEST: %.2f'% r2_score(y_test, y_pred_test))

Coefficient of determination (R2) TRAIN: 0.81
Coefficient of determination (R2) TEST: 0.76


In [9]:
y_pred_test

array([4.36511004, 3.76660705, 5.17902426, 3.31612534, 4.42237486,
       2.73574809, 2.36429798, 2.03871375, 3.00872846, 3.71417147,
       5.85440761, 3.15247154, 3.83083581, 2.9198218 , 3.67109435,
       2.65755914, 2.76813705, 3.64598211, 2.49233394, 4.17056057,
       5.26445845, 4.26856604, 3.9204283 , 2.44655319, 3.58564176,
       2.25491972, 3.77395129, 3.13463789, 2.55964274, 2.46853506,
       3.12986639, 2.61353074, 2.63072618, 1.24803955, 1.97325488,
       3.59559334, 2.41725735, 3.5159711 , 3.63394983, 2.15703043,
       2.65146376, 3.08594658, 4.51924478, 2.34921785, 2.32478469,
       2.80183403, 2.79456781, 3.51940035, 4.48658569, 1.93018607,
       2.03429474, 2.29267517, 2.59485697, 3.03404504, 3.04334462,
       3.86832896, 3.33297477, 3.95212393, 6.75929937, 2.29732534,
       3.23515415, 2.37067481, 3.77904989, 1.11257308, 5.0933627 ,
       1.92183599, 5.40578876, 2.09175397, 5.5978724 , 4.09553099,
       3.2631128 , 3.65322357, 2.30683994, 2.20026973, 2.56579

In [10]:
y_pred_test_series = pd.Series(y_pred_test)
#results_df = pd.DataFrame(data=[y_test, y_pred_test_series], columns=['y_test', 'y_pred_test'])
results_df = pd.DataFrame([y_test, y_pred_test_series])

In [11]:
results_df

,649,208,945,678,802,239,817,527,721,1188,...,333,334,335,336,337,338,340,341,343,345
dollar_price,4.878031,3.513825,4.439742,3.267084,4.795,2.671003,2.303386,1.937935,3.034134,3.596459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed 0,NaN,4.048929,NaN,NaN,NaN,4.251507,NaN,NaN,NaN,NaN,...,2.909422,1.376138,4.66401,3.852041,1.604368,3.325212,3.798454,3.119465,2.295253,1.495526
